In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575516251452_0001,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=livy-session-0>

In [2]:
spark

In [262]:
import pyspark.sql.functions as f
#import pandas as pd
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.mllib.classification import SVMWithSGD
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
import nltk
from nltk.tokenize import word_tokenize

In [71]:
#df = spark.read.json('s3://trendsyelp/review.json')

df = spark.read.option('inferSchema','true').format('json').load('s3://trendsyelp/review.json')

In [72]:
# show schema

df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

In [73]:
df.cache()

DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [87]:
# remove punctuations and 

import string
import re

def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  
    return nopunct

def convert_rating(rating):
    if rating >=4:
        return 1
    else:
        return 0

In [240]:
from pyspark.sql.functions import udf

punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))

df_text = df.select('business_id', 'date', punct_remover('text'), rating_convert('stars')).limit(100)


df_text = df_text.withColumnRenamed('<lambda>(text)', 'text')
df_text = df_text.withColumnRenamed('<lambda>(stars)', 'rate')

In [241]:
df_text.show()

+--------------------+-------------------+--------------------+----+
|         business_id|               date|                text|rate|
+--------------------+-------------------+--------------------+----+
|ujmEBvifdJM6h6RLv...|2013-05-07 04:34:36|Total bill for th...|   0|
|NZnhc2sEQy3RmzKTZ...|2017-01-14 21:30:33|I  adore  Travis ...|   1|
|WTqjgwHlXbSFevF32...|2016-11-09 20:09:03|I have to say tha...|   1|
|ikCg8xy5JIg_NGPx-...|2018-01-09 20:56:38|Went in for a lun...|   1|
|b1b1eb3uo-w561D0Z...|2018-01-30 23:07:38|Today was my seco...|   0|
|eU_713ec6fTGNO4Be...|2013-01-20 13:25:59|I ll be the first...|   1|
|3fw2X5bZYeW9xCz_z...|2016-05-07 01:21:02|Tracy dessert had...|   0|
|zvO-PJCpNk4fgAVUn...|2010-10-05 19:12:35|This place has go...|   0|
|b2jN2mm9Wf3RcrZCg...|2015-01-18 14:04:18|I was really look...|   0|
|oxwGyA17NL6c5t1Et...|2012-02-29 21:52:43|It s a giant Best...|   0|
|8mIrX_LrOnAqWsB5J...|2011-11-30 02:11:15|Like walking back...|   1|
|mRUVMJkUGxrByzMQ2...|2017-12-15 2

In [242]:
df_text.cache()

DataFrame[business_id: string, date: string, text: string, rate: string]

In [243]:
# tolenize the text

from pyspark.ml.feature import *
from pyspark.ml import Pipeline
#tokenizer and stop word remover
tok = Tokenizer(inputCol="text", outputCol="words")
#stop word remover
stopwordrm = StopWordsRemover(inputCol='words', outputCol='words_nsw')
# Build the pipeline 
pipeline = Pipeline(stages=[tok, stopwordrm])
# Fit the pipeline 
review_tokenized = pipeline.fit(df_text).transform(df_text).cache()

In [244]:
review_tokenized.show()

+--------------------+-------------------+--------------------+----+--------------------+--------------------+
|         business_id|               date|                text|rate|               words|           words_nsw|
+--------------------+-------------------+--------------------+----+--------------------+--------------------+
|lPljcKGwLlcwkLT5c...|2011-08-29 17:12:47|The staff pretty ...|   0|[the, staff, pret...|[staff, pretty, m...|
|N0apJkxIem2E8irTB...|2011-07-05 01:11:33|We made the mista...|   1|[we, made, the, m...|[made, mistake, m...|
|XqJG7Ux_mMfMJnyG2...|2018-05-10 18:54:20|I cant give a ful...|   1|[i, cant, give, a...|[cant, give, full...|
|JjupAcmP7-ytPG49j...|2016-02-14 18:45:16|Un service impecc...|   1|[un, service, imp...|[un, service, imp...|
|jOE3XbP3TLPBC8-tv...|2016-10-30 17:36:18|I went to Carolyn...|   1|[i, went, to, car...|[went, carolyn, f...|
|OQ2QdaD0StfptJXv1...|2018-08-27 01:57:44|The place is diff...|   1|[the, place, is, ...|[place, different...|
|

In [245]:
# tri gram

ngram = NGram(inputCol = 'words', outputCol = 'ngram', n = 3)
add_ngram = ngram.transform(review_tokenized)

In [246]:
# count vectorizer and tf-idf

cv_ngram = CountVectorizer(inputCol='ngram', outputCol='tf_ngram')
cvModel_ngram = cv_ngram.fit(add_ngram)
cv_df_ngram = cvModel_ngram.transform(add_ngram)

In [247]:
# create TF-IDF matrix

idf_ngram = IDF().setInputCol('tf_ngram').setOutputCol('tfidf_ngram')
tfidfModel_ngram = idf_ngram.fit(cv_df_ngram)
tfidf_df_ngram = tfidfModel_ngram.transform(cv_df_ngram)

In [248]:
# train test split

splits_ngram = tfidf_df_ngram.select(['tfidf_ngram', 'rate']).randomSplit([0.8,0.2],seed=100)
train_ngram = splits_ngram[0].cache()
test_ngram = splits_ngram[1].cache()

In [249]:
# Convert feature matrix to vectors

train_lb_ngram = train_ngram.rdd.map(lambda row: LabeledPoint(row[1], Vectors.fromML(row[0])))
test_lb_ngram = test_ngram.rdd.map(lambda row: LabeledPoint(row[1], Vectors.fromML(row[0])))

In [252]:
numIterations = 50
regParam = 0.3
svm = SVMWithSGD.train(train_lb_ngram, numIterations, regParam=regParam)

#test_lb = test.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))
scoreAndLabels_test = test_lb_ngram.map(lambda x: (float(svm.predict(x.features)), x.label))
score_label_test = spark.createDataFrame(scoreAndLabels_test, ["prediction_rate", "actual_rate"])

In [253]:
score_label_test.show()

+---------------+-----------+
|prediction_rate|actual_rate|
+---------------+-----------+
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        0.0|
|            1.0|        1.0|
|            1.0|        0.0|
|            1.0|        0.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        0.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            0.0|        0.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            1.0|        1.0|
|            0.0|        0.0|
+---------------+-----------+
only showing top 20 rows

In [256]:
f1_eval = MulticlassClassificationEvaluator(labelCol="actual_rate", predictionCol="prediction_rate", metricName="f1")
svm_f1 = f1_eval.evaluate(score_label_test)
print("F1 score: %.4f" % svm_f1)

F1 score: 0.7925